# Interview Challenge 4: Data Processing Pipeline

## Problem Statement

You are given a dataset of user transactions with the following schema:
- `user_id` (string)
- `transaction_id` (string) 
- `amount` (double)
- `timestamp` (string in 'yyyy-MM-dd HH:mm:ss' format)
- `category` (string)

## Tasks

1. **Data Cleaning**: Remove transactions with null `user_id`, negative amounts, or invalid timestamps
2. **Data Transformation**: 
   - Convert `timestamp` to proper timestamp type
   - Add a `date` column (just the date part)
   - Add a `hour` column (hour of the day)
3. **Aggregations**:
   - Calculate total amount per user per day
   - Calculate average transaction amount per category per hour
4. **Anomaly Detection**: Flag transactions that are 3 standard deviations above the user's daily average

## Requirements
- Use PySpark DataFrame API
- Handle edge cases (empty data, malformed records)
- Optimize for performance where possible
- Include comments explaining your approach

## Sample Data
```python
data = [
    ('user1', 'txn1', 100.0, '2023-01-01 10:30:00', 'shopping'),
    ('user1', 'txn2', 50.0, '2023-01-01 14:20:00', 'food'),
    ('user2', 'txn3', 200.0, '2023-01-01 11:15:00', 'shopping'),
    # ... more data
]
```

## 🚀 Try It Yourself

Implement the solution using the code structure below. Start with data cleaning, then add transformations, aggregations, and finally anomaly detection.

**Tip:** Work through each task step by step. Test your code after each major section.

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create Spark session
spark = SparkSession.builder.appName("InterviewChallenge4").getOrCreate()

# Sample data (in real scenario, this would be read from file)
data = [
    ('user1', 'txn1', 100.0, '2023-01-01 10:30:00', 'shopping'),
    ('user1', 'txn2', 50.0, '2023-01-01 14:20:00', 'food'),
    ('user2', 'txn3', 200.0, '2023-01-01 11:15:00', 'shopping'),
    ('user1', 'txn4', -10.0, '2023-01-01 16:45:00', 'invalid'),  # negative amount
    (None, 'txn5', 75.0, '2023-01-01 12:00:00', 'food'),  # null user_id
    ('user2', 'txn6', 300.0, '2023-01-01 09:30:00', 'shopping'),
]

# Define schema
schema = StructType([
    StructField('user_id', StringType(), True),
    StructField('transaction_id', StringType(), True),
    StructField('amount', DoubleType(), True),
    StructField('timestamp', StringType(), True),
    StructField('category', StringType(), True)
])

# Create DataFrame
df = spark.createDataFrame(data, schema)
df.show()

# === YOUR SOLUTION GOES HERE ===
# Implement the 4 tasks below

# Task 1: Data Cleaning
# Remove transactions with null user_id, negative amounts, or invalid timestamps

# Task 2: Data Transformation
# Convert timestamp to proper format, add date and hour columns

# Task 3: Aggregations
# Calculate total amount per user per day
# Calculate average transaction amount per category per hour

# Task 4: Anomaly Detection
# Flag transactions that are 3 standard deviations above user's daily average

# Display your results
print("Implementation completed!")


## ✅ Solution Reference

**⚠️ Only refer to this after attempting the problem yourself!**

Here's a complete solution implementation. Compare it with your approach and understand the differences.

In [ ]:
# SOLUTION IMPLEMENTATION

# 1. Data Cleaning
print("1. Data Cleaning:")
cleaned_df = df.filter(
    (col('user_id').isNotNull()) & 
    (col('amount') > 0) & 
    (col('timestamp').rlike(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}'))
)
cleaned_df.show()
print(f"Original records: {df.count()}, Cleaned records: {cleaned_df.count()}")

# 2. Data Transformation
print("\n2. Data Transformation:")
transformed_df = cleaned_df.withColumn(
    'parsed_timestamp', to_timestamp('timestamp')
).withColumn(
    'date', date_format('parsed_timestamp', 'yyyy-MM-dd')
).withColumn(
    'hour', hour('parsed_timestamp')
).drop('timestamp').withColumnRenamed('parsed_timestamp', 'timestamp')

transformed_df.show()

# 3. Aggregations
print("\n3. Aggregations:")

# Total amount per user per day
user_daily_totals = transformed_df.groupBy('user_id', 'date').agg(
    sum('amount').alias('daily_total'),
    count('transaction_id').alias('transaction_count')
)
print("User Daily Totals:")
user_daily_totals.show()

# Average transaction amount per category per hour
category_hourly_avg = transformed_df.groupBy('category', 'hour').agg(
    avg('amount').alias('avg_amount'),
    count('transaction_id').alias('transaction_count')
)
print("Category Hourly Averages:")
category_hourly_avg.show()

# 4. Anomaly Detection
print("\n4. Anomaly Detection:")

# Calculate daily averages per user
user_daily_stats = transformed_df.groupBy('user_id', 'date').agg(
    avg('amount').alias('daily_avg'),
    stddev('amount').alias('daily_stddev'),
    count('transaction_id').alias('daily_count')
)

# Join back and flag anomalies (3 standard deviations above daily average)
anomalies_df = transformed_df.join(
    user_daily_stats,
    ['user_id', 'date']
).withColumn(
    'is_anomaly',
    when(
        (col('amount') > col('daily_avg') + 3 * col('daily_stddev')) & 
        (col('daily_stddev') > 0),  # Avoid division by zero
        True
    ).otherwise(False)
).select(
    'user_id', 'transaction_id', 'amount', 'date', 'hour', 'category',
    'daily_avg', 'daily_stddev', 'is_anomaly'
)

print("Anomaly Detection Results:")
anomalies_df.show()

print(f"\nTotal anomalies detected: {anomalies_df.filter('is_anomaly = true').count()}")

print("\n✅ Solution completed! Review the results and understand each step.")